In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import io
import urllib.request
import datetime
import gzip
import geopandas as gpd
import os
import zipfile as zf
import shapefile
from shapely.geometry import shape

Importing file with world country borders as coordinates

In [2]:
zp = zf.ZipFile('TM_WORLD_BORDERS-0.3.zip')
files_to_read = [y for y in zp.namelist() for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)]
dummy = zp.read(files_to_read[0])
dbf_file, prj_file, shp_file, shx_file = [zp.open(filename) for filename in files_to_read]
r = shapefile.Reader(shp = shp_file, shx = shx_file, dbf = dbf_file, encoding='windows-1252')
attributes, geometry = [], []
field_names = [field[0] for field in r.fields[1:]]
for row in r.shapeRecords():
    geometry.append(shape(row.shape.__geo_interface__))
    attributes.append(dict(zip(field_names, row.record)))
#Creating a GeoDataframe of the World Borders
gdf = gpd.GeoDataFrame(data = attributes, geometry = geometry)

Scraping temperature anomlalies from NASA dataset

In [3]:
url = "https://data.giss.nasa.gov/pub/gistemp/gistemp1200_GHCNv4_ERSSTv5.nc.gz"
req = urllib.request.Request(url)
with gzip.open(urllib.request.urlopen(req)) as resp:
    xr_df = xr.open_dataset(io.BytesIO(resp.read()))
dfnasa = xr_df.to_dataframe()
#Transforming into pandas dataframe
dfnasa = dfnasa.reset_index()

In [7]:
dfnasa.head()

,lat,lon,nv,time,time_bnds,tempanomaly
0,-89.0,-179.0,0,1880-01-15,1880-01-01,NaN
1,-89.0,-179.0,0,1880-02-15,1880-02-01,NaN
2,-89.0,-179.0,0,1880-03-15,1880-03-01,NaN
3,-89.0,-179.0,0,1880-04-15,1880-04-01,NaN
4,-89.0,-179.0,0,1880-05-15,1880-05-01,NaN


Set year from which to obtain data and country

In [13]:
year = 1980
iso3= "USA"

In [12]:
def tempdata(year, iso3):
    #Subsetting the year range from the provided year
    tempyear = dfnasa[dfnasa['time'].dt.year >= year]
    #Creating a GeoDataFrame
    tempyear = gpd.GeoDataFrame(tempyear, geometry=gpd.points_from_xy(tempyear.lon, tempyear.lat))
    #Merging with the border data
    bord = gpd.sjoin(gdf, tempyear, how="inner")
    yearcountry = bord[bord["ISO3"]==iso3]
    #Sorting values by time
    yearcountry = yearcountry.sort_values(by=["time"])
    yearcountry = pd.DataFrame(yearcountry)
    yearcountry = yearcountry.drop(columns= ["FIPS", "UN", "AREA","POP2005", "index_right", "nv"])
    yearcountry["YearMonth"]=yearcountry["time"].dt.strftime("%Y-%m")
    del tempyear
    return yearcountry

In [14]:
df = tempdata(year,iso3)

Export to pickle

In [18]:
df.to_pickle("temperature_anomalies_"+str(year)+"_"+country+".pkl")